# Distributed IO

We'll see that for applications that are limited by IO bandwidth, a wide distribution across compute nodes can be beneficial.

## Technical preamble

Spin up a Jobqueue cluster that has 6 workers on 6 different nodes.
(We'll ensure different nodes for each job by requesting more than 50% of the available CPUs in each job.)

In [1]:
import dask, dask.distributed
import dask_jobqueue

# NQS != PBS:
dask.config.set({'jobqueue.pbs.walltime': None})

cluster = dask_jobqueue.PBSCluster(

    # Dask workers
    cores=16, memory='96GB',
    processes=1, # workers per job

    # PBS job script
    queue='cltestque', name='dask-worker',
    resource_spec=('elapstim_req=00:45:00,'
                   'cpunum_job=17,memsz_job=96gb'),
    interface='ib0', local_directory='/scratch',

)

client = dask.distributed.Client(cluster)
cluster.scale(jobs=6)

In [2]:
client

Client Scheduler: tcp://192.168.31.162:38588 Dashboard: http://192.168.31.162:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Create random data and write them to disk

In [3]:
from dask import array as darr

In [4]:
random_data = darr.random.normal(
    size=(int(100_000_000_000 / 8), ),
    chunks=(int(500_000_000 / 8), )
)
random_data

,Array,Chunk
Bytes,100.00 GB,500.00 MB
Shape,"(12500000000,)","(62500000,)"
Count,200 Tasks,200 Chunks
Type,float64,numpy.ndarray


In [5]:
!rm -rf random_data.zarr/

In [6]:
%time random_data.to_zarr("random_data.zarr")

CPU times: user 2.52 s, sys: 301 ms, total: 2.82 s
Wall time: 1min 1s


In [7]:
!du -sh random_data.zarr/

89G	random_data.zarr/


## Find largest number with disk IO

We'll re-read the data and find the maximum on the fly.

Note in the Dask dashboard that we don't saturate CPU load.
This means we're limited by IO rather than compute.

In [8]:
random_data = darr.from_zarr("random_data.zarr/")
random_data

,Array,Chunk
Bytes,100.00 GB,500.00 MB
Shape,"(12500000000,)","(62500000,)"
Count,201 Tasks,200 Chunks
Type,float64,numpy.ndarray


In [9]:
%time random_data.max().compute()

CPU times: user 1.23 s, sys: 87.1 ms, total: 1.31 s
Wall time: 15.4 s


6.179690081242766

We've just read and digested 90GB from disk, decompressed it to 100GB and found the maximum in 15 seconds.

That's approx. 6 GB/s.

## Decrease cluster size and see effect on IO bandwidth

In [10]:
cluster.scale(jobs=1)

In [11]:
!qstat

RequestID       ReqName  UserName Queue     Pri STT S   Memory      CPU   Elapse R H M Jobs
--------------- -------- -------- -------- ---- --- - -------- -------- -------- - - - ----
362233.nesh-bat jupyterl smomw122 cltestqu    0 RUN -  243.28M    35.58     5392 Y Y Y    1 
362641.nesh-bat dask-wor smomw122 cltestqu    0 RUN -  219.20M   250.08      119 Y Y Y    1 


In [12]:
client

Client Scheduler: tcp://192.168.31.162:38588 Dashboard: http://192.168.31.162:8787/status,Cluster Workers: 1 Cores: 16 Memory: 96.00 GB


In [13]:
random_data = darr.from_zarr("random_data.zarr/")
random_data

,Array,Chunk
Bytes,100.00 GB,500.00 MB
Shape,"(12500000000,)","(62500000,)"
Count,201 Tasks,200 Chunks
Type,float64,numpy.ndarray


In [14]:
%time random_data.max().compute()

CPU times: user 3.2 s, sys: 280 ms, total: 3.48 s
Wall time: 50.6 s


6.179690081242766

We've just read and digested 90GB from disk, decompressed it to 100GB and found the maximum in 50 seconds.

That's approx. 2 GB/s.

## Bottom line

For IO bound problems, we'd like to be able to scale horizontally rather than vertically.

That's something that could be tackled with the scheduler config (fill all nodes equally vs. keep as many nodes as possible empty).